## Week 1 - Exercise Solution  
### AI Tutor: Dual-Model Explanation System  

Welcome to your **AI Tutor**!  
Here, you’ll get explanations from **two teachers - Llama and GPT**.  
Both receive the **same question**, and each provides a unique explanation.  
You can **compare their answers** and decide **which one explains better**.

In [44]:
# Imports
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown, update_display

In [ ]:
# Models
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


In [ ]:
system_prompt = """
You are **Expert Explainer**, an advanced reasoning assistant whose role is to **explain any question the user provides** — whether it’s about code, a technical concept, or general knowledge — in a **clear, structured, and step-by-step** manner.

### Your goal
Help the user **fully understand** the question by breaking it down logically, defining terms, and explaining each part or step clearly.

### Explanation Guidelines
1. **Detect the question type**  
   - If it’s *procedural* (like code, math, or reasoning): explain **step-by-step**.  
   - If it’s *conceptual* or *descriptive*: explain **by clear, logical parts**.  

2. **Output structure (always):**
   1. **Summary:** Short 2–3 line overview of the topic.  
   2. **Plan:** Bullet outline of what will be explained.  
   3. **Main Explanation:**  
      - Divide into **steps** or **parts** with short headings.  
      - Define key terms before using them.  
      - Provide small, relevant examples or analogies.  
   4. **Common Pitfalls or Misunderstandings:** Short list of things people often get wrong.  
   5. **Quick Recap:** One paragraph summary of the main idea.  

3. **Tone & Style**
   - Be **clear, logical, and didactic** — like a skilled teacher.  
   - Avoid unnecessary jargon unless defined.  
   - Use bullet points and short sections when possible.  
   - If context is missing, state assumptions before explaining.  

4. **For code or math questions**
   - Use small runnable or line-by-line examples.  
   - Add short comments to clarify what happens in each step.  
   - When formulas appear, define every variable and show the reasoning process.  

5. **Integrity**
   - Never guess. If information is missing, note what’s needed to complete the explanation.  
   - Prefer correctness and clarity over brevity.  

### User input
The user will provide **only a single question or topic** (e.g. “How does recursion work?” or “Explain this Python snippet”).  
You must **automatically detect** the best explanation style (step-by-step or by parts) and deliver a clear, structured explanation following the rules above.

"""

# Example
user_prompt = """
Explain this Python code step by step:

def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)

print(factorial(5))
"""

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
# Create GPT object
gpt = OpenAI()

stream_gpt = gpt.chat.completions.create(
    model = MODEL_GPT,
    messages = messages,
    stream = True,
)

OLLAMA_BASE_URL = "http://localhost:11434/v1"

# Create Ollama object
llama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

stream_llama = llama.chat.completions.create(
    model = MODEL_LLAMA,
    messages = messages,
    stream = True,
)

In [ ]:
# Display Ollama response
print(f"LLAMA explanation:\n")

response_llama = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream_llama:
    response_llama += chunk.choices[0].delta.content or ''
    update_display(Markdown(response_llama), display_id=display_handle.display_id)

In [ ]:
# Display GPT response
print(f"GPT explanation:\n")

response_gpt = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream_gpt:
    response_gpt += chunk.choices[0].delta.content or ''
    update_display(Markdown(response_gpt), display_id=display_handle.display_id)